The plan is to implement PSO in order to find the different optima from our parameter space. As PSO needs an evaluation function to work, we will use a linear regression model to predict the speed based on the parameters being tried. 

The linear regression is done from the collected data from the robot.

### Imports

In [1]:
# Imports
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, mean_squared_error
from sklearn.ensemble import RandomForestRegressor

import copy
import numpy.random as rnd
import time
import matplotlib.pyplot as plt

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import random
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import animation

### Regression Models

In [2]:
# Get the data
df = pd.read_csv("firstbatch_500Samples.csv")

In [3]:
df.head()

,Unnamed: 0,Snelheid,Omvang1,Positie1,Omvang2,Positie2,Relatie,Speed
0,0,0.0,0.0,-15.0,15.0,-75.0,15.0,1.012087
1,1,0.0,0.0,15.0,30.0,30.0,-90.0,0.844846
2,2,0.0,0.0,15.0,60.0,-30.0,-90.0,2.491373
3,3,0.0,0.0,45.0,60.0,75.0,60.0,1.249660
4,4,0.0,0.0,75.0,30.0,-75.0,-90.0,1.112912


In [4]:
# Prepare the data for the model

X = df.drop(columns = ['Speed', 'Unnamed: 0'], axis = 1).values
y = df.Speed.values

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 18)

# Create the model
model = LinearRegression()

# Train it
model.fit(X_train, y_train)

# Make prediction
y_pred = model.predict(X_test)
#  Evaluate the model
print('R^2 score: ', model.score(X_test, y_test))
print('MSE : ', mean_squared_error(y_test, y_pred))

R^2 score:  0.5785560150799474
MSE :  0.7903636488460546


In [5]:
# Regression Tree
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score

regressor = make_pipeline(StandardScaler(), DecisionTreeRegressor(random_state=0))
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)

print('R^2 score: ',regressor.score(X_test, y_test))
print('MSE : ', mean_squared_error(y_test, y_pred))

R^2 score:  0.20513749661448355
MSE :  1.4906617509936955


In [6]:
# SVR
from sklearn.svm import SVR
import numpy as np

regr = make_pipeline(StandardScaler(), SVR(C=1.0, epsilon=0.2, kernel = 'rbf'))
regr.fit(X_train, y_train)

y_pred = regr.predict(X_test)

print('R^2 score: ',regr.score(X_test, y_test))
print('MSE : ', mean_squared_error(y_test, y_pred))

R^2 score:  0.6629838885299169
MSE :  0.632030099164739


In [7]:
# Random forest Regressor

regr = RandomForestRegressor(max_depth=6, random_state=42)
regr.fit(X_train, y_train)
y_pred = regr.predict(X_test)

print('R^2 score: ',regr.score(X_test, y_test))
print('MSE : ', mean_squared_error(y_test, y_pred))

R^2 score:  0.6464146981732082
MSE :  0.6631034712315804


Export the function to be optimized

In [8]:
print(model.intercept_, model.coef_)
intercept = model.intercept_
coefs = model.coef_

-0.1830203116455449 [ 3.42065695e-01  2.51984365e-02 -2.55927956e-03  2.10004768e-02
  1.56109176e-03 -5.05334352e-05]


### PSO

In [11]:
def fitness_function(model, parameters):

  # intercept = model.intercept_
  # coefs = model.coef_

  # sum = 0  
  # for i in range(6):
  #   sum += (coefs[i] * parameters[i])
  # f = intercept + sum
  return model.predict(np.array(parameters).reshape(1, 6))

In [19]:
def update_velocity(particle, velocity, pbest, gbest, w_min=0.5, max=1.0, c=0.1):

  # Initialise new velocity array
  num_particle = len(particle)
  new_velocity = np.array([0.0 for i in range(num_particle)])

  # Randomly generate r1, r2 and inertia weight from normal distribution
  r1 = random.uniform(0,max)
  r2 = random.uniform(0,max)
  w = random.uniform(w_min,max)
  c1 = c
  c2 = c
  # Calculate new velocity
  for i in range(num_particle):
    new_velocity[i] = w*velocity[i] + c1*r1*(pbest[i]-particle[i])+c2*r2*(gbest[i]-particle[i])

  return new_velocity

In [20]:
def update_position(particle, velocity):

  bounds = [[-90, 90], [-90, 90], [0, 1], [0, 90], [0, 90], [-90, 90]]

  new_particle = np.zeros(6)

  # print('b', particle)
  # print('v', velocity)
  # Move particles by adding velocity
  for i in range(6):
    if (particle[i] + velocity[i]) >= bounds[i][0] and (particle[i] + velocity[i]) <= bounds[i][1]:
      new_particle[i] = particle[i] + velocity[i]
    
  # print('a', new_particle)
  return new_particle

In [21]:
def pso(population_size, dimension, parameters, generation, fitness_criterion, model):
  # Initialisation
  # Population

  particles = []

  for i in range(population_size):
    # Create a particle and add it to the population
    particle = []
    for j in range(6):
      # Populate the particle with 6 random number from the 6 parameters
      lower_bound = list(parameters.values())[j][0]
      upper_bound = list(parameters.values())[j][1]

      # Some parameters have to be integers
      if j == 2:
        particle.append(random.randint(lower_bound, upper_bound)/10)
      else:
        particle.append(random.randint(lower_bound, upper_bound))

    particles.append(particle)

  #particles = [[random.uniform(position_min, position_max) for j in range(dimension)] for i in range(population)]
  # Particle's best position
  pbest_position = particles
  # Fitness
  pbest_fitness = []

  for particle in particles:
    fitness = fitness_function(model, particle)
    pbest_fitness.append(fitness)

  # Index of the best particle
  gbest_index = np.argmin(pbest_fitness)
  # Global best particle position
  gbest_position = pbest_position[gbest_index]
  # print(gbest_position)
  # Velocity (starting from 0 speed)
  velocity = [[0.0 for j in range(dimension)] for i in range(population_size)]
  
  # Loop for the number of generation
  for t in range(generation):
    # Stop if the average fitness value reached a predefined success criterion
    if np.average(pbest_fitness) <= fitness_criterion:
      break
    else:
      for n in range(population_size):
        # Update the velocity of each particle
        velocity[n] = update_velocity(particles[n], velocity[n], pbest_position[n], gbest_position)
        # Move the particles to new position
        particles[n] = update_position(particles[n], velocity[n])
        
    # Calculate the fitness value
    pbest_fitness = [fitness_function(model, particle) for particle in particles]
    # Find the index of the best particle
    gbest_index = np.argmin(pbest_fitness)
    # Update the position of the best particle
    gbest_position = pbest_position[gbest_index]
    # print(gbest_position)

  # Print the results
  print('Global Best Position: ', gbest_position)
  print('Best Fitness Value: ', min(pbest_fitness))
  print('Average Particle Best Fitness Value: ', np.average(pbest_fitness))
  print('Number of Generation: ', t)


In [22]:
population_size = 1000
dimension = 6
generation = 1000
fitness_criterion = 10e-4

In [23]:
parameters = {
    'Angles_0' : [-90, 90],
    'Angles_1' : [-90, 90],
    'freq' : [0, 10],
    'amp_1' : [0, 90],
    'amp_2' : [0, 90],
    'phase' : [-90, 90]
}

In [24]:
pso(population_size, dimension, parameters, generation, fitness_criterion, model)

Global Best Position:  [-88. -73.   0.   0.   8. -64.]
Best Fitness Value:  -32.108564439061006
Average Particle Best Fitness Value:  -0.7845975820355868
Number of Generation:  1


# New trial

In [9]:
def check_constraints(temp, bounds):

  counter = 0

  for i in range(6):
    if temp[i] >= bounds[i][0] and temp[i] <= bounds[i][1] :
      counter += 1

  return counter == 6

In [13]:
import random

# Define the problem-specific parameters
num_particles = 20
max_iterations = 100
search_space = [[-90, 90], [-90, 90], [0, 1], [0, 90], [0, 90], [-90, 90]] # Define the search space for each parameter
inertia = 0.5
cognitive_constant = 1.5
social_constant = 1.5


# Initialize the swarm
swarm = []
best_positions = []
global_best_position = None
global_best_fitness = float('inf')

for _ in range(num_particles):
    # Initialize particle position and velocity randomly within the search space
    position = [random.uniform(low, high) for low, high in search_space]
    velocity = [random.uniform(-1, 1) for _ in range(6)]
    
    # Evaluate fitness
    fitness = fitness_function(regr, position)
    
    # Initialize personal best position and fitness
    personal_best_position = position
    personal_best_fitness = fitness
    
    # Update global best position and fitness
    if fitness < global_best_fitness:
        global_best_position = position
        global_best_fitness = fitness
    
    # Add particle to the swarm
    swarm.append((position, velocity, personal_best_position, personal_best_fitness))
    best_positions.append(personal_best_position)

# PSO main loop
iteration = 0
while iteration < max_iterations:
    for i in range(num_particles):
        # Update particle velocity and position if constraints are met
        position, velocity, personal_best_position, _ = swarm[i]
        
        # Update velocity
        velocity = (inertia * np.array(velocity) + cognitive_constant * random.uniform(0, 1) * (np.array(personal_best_position) - np.array(position)) + social_constant * random.uniform(0, 1) * (np.array(personal_best_position) - np.array(position)))
        
        # Update position
        position_temp = np.array(position) + np.array(velocity)
        
        if check_constraints(position_temp, search_space):
            position = position_temp

        # Evaluate fitness
        fitness = fitness_function(regr, position)
        
        # Update personal best position and fitness
        if fitness < swarm[i][3]:
            swarm[i] = (position, velocity, position, fitness)
            best_positions[i] = position
        
        # Update global best position and fitness
        if fitness > global_best_fitness:
            global_best_position = position
            global_best_fitness = fitness
    
    iteration += 1

# Retrieve the optimized solution
optimized_solution = global_best_position

# Print or process the optimized solution
print("Optimized solution:", optimized_solution)


Optimized solution: [ 54.45816511  69.99680704   0.73888297  63.40852966  87.89192915
 -39.74997398]


In [14]:
regr.predict(np.array(optimized_solution).reshape(1, 6))

array([6.06078476])